In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data-p3/joined_all.csv')
df.head()


,Pass_ID,Sch_Departure,Act_Departure,C0 - S2,C_Start,C_Avg,S2,Wait_Time,Day_of_Week,Period_of_Week,Airfield,Season
0,5368296,2028-09-03 13:12,2028-09-03 13:12,1,1,1.0,2028-09-03 11:26,3.0,7 - SUN,2 - WEEKEND,AUC,3 - SUMMER
1,5480057,2028-09-15 9:36,2028-09-15 9:36,1,1,1.0,2028-09-15 7:45,3.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER
2,5349077,2028-09-01 10:21,2028-09-01 11:09,1,1,1.0,2028-09-01 7:58,5.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER
3,5349207,2028-09-01 10:21,2028-09-01 11:09,1,1,1.0,2028-09-01 8:04,4.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER
4,5349308,2028-09-01 10:21,2028-09-01 11:09,1,1,1.0,2028-09-01 8:13,7.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER


In [ ]:
df['S2'] = pd.to_datetime(df['S2'])
# df['wait_time_delta'] = pd.to_timedelta(df['Wait_Time'], unit='m')
# df['S1'] =df['S2'] - df['Wait_Time']

UFuncTypeError: ufunc 'subtract' cannot use operands with types dtype('<M8[ns]') and dtype('float64')

In [4]:
def determine_cluster(timestamp):

    is_weekend = timestamp.weekday() >= 5  
    hour = timestamp.hour
    
    if 0 <= hour < 4:
        cluster = "0:00 - 4:00"
    elif 4 <= hour < 8:
        cluster = "4:00 - 8:00"
    elif 8 <= hour < 12:
        cluster = "8:00 - 12:00"
    elif 12 <= hour < 16:
        cluster = "12:00 - 16:00"
    elif 16 <= hour < 20:
        cluster = "16:00 - 20:00"
    else:
        cluster = "20:00 - 0:00"
    
    return f"{'Weekend' if is_weekend else 'Weekday'} - {cluster}"

df['cluster'] = df['S2'].apply(determine_cluster)

In [34]:
def impute_exponential_by_cluster(df, cluster_col='cluster', target_col='Wait_Time', time_col='S2'):
    df_imputed = df.copy()
    
    # Calculate means for each cluster
    cluster_means = df.groupby(cluster_col)[target_col].mean()
    
    # Function to find an existing wait time or sample from exponential
    def impute_value(row):
        if pd.isna(row[target_col]):  # If value is missing
            cluster = row[cluster_col]
            arrival_time = row[time_col]
            
            # Check if someone else in the same cluster and arrival time has a Wait_Time
            same_time_value = df_imputed[
                (df_imputed[cluster_col] == cluster) &
                (df_imputed[time_col] == arrival_time) &
                (~pd.isna(df_imputed[target_col]))
            ][target_col]
            
            if not same_time_value.empty:  # If a valid Wait_Time exists, use it
                return same_time_value.iloc[0]
            
            # Otherwise, sample from exponential
            scale = cluster_means[cluster]  # Mean of the feature for the cluster
            if scale > 0:  # Ensure the scale is valid
                return np.random.exponential(scale=scale)
        
        # Return original value if not missing
        return row[target_col]
    
    # Apply the imputation
    df_imputed['wait_time_imp'] = df_imputed.apply(impute_value, axis=1)
    return df_imputed


In [35]:
df = impute_exponential_by_cluster(df)

/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/2439256362.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cluster_means = df.groupby(cluster_col)[target_col].mean()


In [33]:
cluster_ord = [
    "Weekday - 0:00 - 4:00", "Weekday - 4:00 - 8:00", "Weekday - 8:00 - 12:00", 
    "Weekday - 12:00 - 16:00", "Weekday - 16:00 - 20:00", "Weekday - 20:00 - 0:00",
    "Weekend - 0:00 - 4:00", "Weekend - 4:00 - 8:00", "Weekend - 8:00 - 12:00", 
    "Weekend - 12:00 - 16:00", "Weekend - 16:00 - 20:00", "Weekend - 20:00 - 0:00"
]

df['cluster'] = pd.Categorical(df['cluster'], categories=cluster_ord, ordered=True)
df = df.sort_values('cluster')

In [36]:
df["S1_imp"] = df['S2'] - pd.to_timedelta(df['wait_time_imp'])
# df["cluster_S1"] = df['S1_imp'].apply(determine_cluster)

In [37]:
def cluster_hours(timestamp):
    is_weekend = timestamp.weekday() >= 5

    weekday = (153 - 44) * 4
    weekend = 44 * 4

    
    return weekend if is_weekend else weekday

df['cluster_hours'] = df['S2'].apply(cluster_hours)

In [38]:
df[['cluster','cluster_hours']]

,cluster,cluster_hours
57071,Weekday - 4:00 - 8:00,436
56736,Weekday - 4:00 - 8:00,436
56735,Weekday - 4:00 - 8:00,436
91991,Weekday - 4:00 - 8:00,436
91992,Weekday - 4:00 - 8:00,436
...,...,...
15664,Weekend - 20:00 - 0:00,176
15665,Weekend - 20:00 - 0:00,176
15666,Weekend - 20:00 - 0:00,176
15661,Weekend - 20:00 - 0:00,176


In [39]:
arrival_counts = df.groupby('cluster').size()
arrival_counts.head()


/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/253566319.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  arrival_counts = df.groupby('cluster').size()


cluster
Weekday - 0:00 - 4:00          0
Weekday - 4:00 - 8:00      15203
Weekday - 8:00 - 12:00     20785
Weekday - 12:00 - 16:00    21850
Weekday - 16:00 - 20:00    20992
dtype: int64

In [40]:
results = arrival_counts.to_frame(name='count')

results = results.merge(df[['cluster', 'cluster_hours']].drop_duplicates(), on='cluster', how='left')
results.head()


,cluster,count,cluster_hours
0,Weekday - 0:00 - 4:00,0,NaN
1,Weekday - 4:00 - 8:00,15203,436.0
2,Weekday - 8:00 - 12:00,20785,436.0
3,Weekday - 12:00 - 16:00,21850,436.0
4,Weekday - 16:00 - 20:00,20992,436.0


In [ ]:
results['avg_arrival_rate'] = results['count'] / results['cluster_hours'] / 60 # per minute
results


,cluster,count,cluster_hours,avg_arrival_rate
0,Weekday - 0:00 - 4:00,0,NaN,NaN
1,Weekday - 4:00 - 8:00,15203,436.0,0.581154
2,Weekday - 8:00 - 12:00,20785,436.0,0.794534
3,Weekday - 12:00 - 16:00,21850,436.0,0.835245
4,Weekday - 16:00 - 20:00,20992,436.0,0.802446
5,Weekday - 20:00 - 0:00,1546,436.0,0.059098
6,Weekend - 0:00 - 4:00,0,NaN,NaN
7,Weekend - 4:00 - 8:00,5973,176.0,0.565625
8,Weekend - 8:00 - 12:00,7472,176.0,0.707576
9,Weekend - 12:00 - 16:00,9494,176.0,0.899053


In [43]:
average_C0 = df.groupby('cluster')['C0 - S2'].mean().reset_index(name='avg_C0')
average_C0

/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/441160710.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_C0 = df.groupby('cluster')['C0 - S2'].mean().reset_index(name='avg_C0')


,cluster,avg_C0
0,Weekday - 0:00 - 4:00,NaN
1,Weekday - 4:00 - 8:00,1.830626
2,Weekday - 8:00 - 12:00,1.264037
3,Weekday - 12:00 - 16:00,1.235103
4,Weekday - 16:00 - 20:00,1.372237
5,Weekday - 20:00 - 0:00,1.021992
6,Weekend - 0:00 - 4:00,NaN
7,Weekend - 4:00 - 8:00,1.776996
8,Weekend - 8:00 - 12:00,1.154042
9,Weekend - 12:00 - 16:00,1.203076


In [44]:
distribution_C0 = pd.crosstab(df['cluster'], df['C0 - S2'], normalize='index') * 100
distribution_C0.columns = [f'{col}_total_servers' for col in distribution_C0.columns]
distribution_C0


,0_total_servers,1_total_servers,2_total_servers,3_total_servers
cluster,,,,
Weekday - 4:00 - 8:00,0.000000,20.384135,76.169177,3.446688
Weekday - 8:00 - 12:00,0.000000,74.577821,24.440702,0.981477
Weekday - 12:00 - 16:00,0.000000,77.066362,22.356979,0.576659
Weekday - 16:00 - 20:00,0.000000,63.933880,34.908537,1.157584
Weekday - 20:00 - 0:00,0.064683,97.671410,2.263907,0.000000
Weekend - 4:00 - 8:00,0.000000,24.962330,72.375691,2.661979
Weekend - 8:00 - 12:00,0.000000,84.595824,15.404176,0.000000
Weekend - 12:00 - 16:00,0.000000,79.692437,20.307563,0.000000
Weekend - 16:00 - 20:00,0.000000,57.371104,38.698294,3.930602


In [17]:
results = average_C0.merge(distribution_C0, on='cluster', how='left')
results

,cluster,avg_C0,0_total_servers,1_total_servers,2_total_servers,3_total_servers
0,Weekday - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN
1,Weekday - 4:00 - 8:00,1.830626,0.000000,20.384135,76.169177,3.446688
2,Weekday - 8:00 - 12:00,1.264037,0.000000,74.577821,24.440702,0.981477
3,Weekday - 12:00 - 16:00,1.235103,0.000000,77.066362,22.356979,0.576659
4,Weekday - 16:00 - 20:00,1.372237,0.000000,63.933880,34.908537,1.157584
5,Weekday - 20:00 - 0:00,1.021992,0.064683,97.671410,2.263907,0.000000
6,Weekend - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN
7,Weekend - 4:00 - 8:00,1.776996,0.000000,24.962330,72.375691,2.661979
8,Weekend - 8:00 - 12:00,1.154042,0.000000,84.595824,15.404176,0.000000
9,Weekend - 12:00 - 16:00,1.203076,0.000000,79.692437,20.307563,0.000000


In [ ]:
# idk if different days makes sense anymore, I looked at the video again and he said the significant trends were based on weekday and weekend

# def imputation_cluster(timestamp):

#     day_name = timestamp.strftime('%A')  # day of week
#     hour = timestamp.hour
    
#     if 0 <= hour < 4:
#         time_cluster = "0:00 - 4:00"
#     elif 4 <= hour < 8:
#         time_cluster = "4:00 - 8:00"
#     elif 8 <= hour < 12:
#         time_cluster = "8:00 - 12:00"
#     elif 12 <= hour < 16:
#         time_cluster = "12:00 - 16:00"
#     elif 16 <= hour < 20:
#         time_cluster = "16:00 - 20:00"
#     else:
#         time_cluster = "20:00 - 0:00"
    

#     return f"{day_name} - {time_cluster}"

# df['imputation_cluster'] = df['S2'].apply(imputation_cluster)


In [ ]:
# imp_avg_wait = df.groupby('imputation_cluster')['Wait_Time'].mean()

# df['wait_time_imp'] = df['Wait_Time'].fillna(df['imputation_cluster'].map(imp_avg_wait))
# df['wait_time_imp'].hasnans


False

In [45]:
df['wait_time_imp']

57071     3.000000
56736     6.000000
56735     9.000000
91991     4.366075
91992    15.000000
           ...    
15664     3.000000
15665     3.000000
15666     3.000000
15661     1.000000
46802     2.000000
Name: wait_time_imp, Length: 114143, dtype: float64

In [46]:
#df['wait_time_imp'] = df['wait_time_imp'] / pd.Timedelta(minutes=1) # makes integer

thresholds = [5, 10, 15, 20, 25, 30]

results = {}

for cluster, group in df.groupby('cluster'):
    results[cluster] = {f"<= {threshold} mins": (group['wait_time_imp'] <= threshold).mean() * 100
                        for threshold in thresholds}

percentage_table_by_cluster = pd.DataFrame(results).transpose()
percentage_table_by_cluster.index.name = 'cluster'
percentage_table_by_cluster.reset_index(inplace=True)
percentage_table_by_cluster


/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/1331756779.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for cluster, group in df.groupby('cluster'):


,cluster,<= 5 mins,<= 10 mins,<= 15 mins,<= 20 mins,<= 25 mins,<= 30 mins
0,Weekday - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN,NaN
1,Weekday - 4:00 - 8:00,45.306847,73.189502,88.028679,95.816615,98.618694,99.447477
2,Weekday - 8:00 - 12:00,60.250180,82.496993,92.090450,96.680298,98.349771,99.494828
3,Weekday - 12:00 - 16:00,70.274600,90.736842,97.052632,99.093822,99.510297,99.766590
4,Weekday - 16:00 - 20:00,77.948742,94.636052,98.589939,99.556974,99.857088,99.938072
5,Weekday - 20:00 - 0:00,93.208279,99.353169,99.935317,100.000000,100.000000,100.000000
6,Weekend - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN,NaN
7,Weekend - 4:00 - 8:00,56.437301,86.053909,97.170601,99.447514,99.933032,99.933032
8,Weekend - 8:00 - 12:00,68.388651,90.390792,96.654176,98.808887,99.571734,99.933084
9,Weekend - 12:00 - 16:00,70.644618,91.752686,98.325258,99.768275,99.957868,99.957868


In [47]:
cluster_metrics = df.groupby('cluster').apply(
    lambda group: pd.Series({
        'death_rate': len(group) / (group['cluster_hours'].min() * 60),
        'avg_wait': group['wait_time_imp'].mean()
    })
).reset_index()

cluster_metrics['arrival_rate'] = (cluster_metrics['avg_wait'] * ((cluster_metrics['death_rate'])**2)) / (1 + cluster_metrics['death_rate'] * cluster_metrics['avg_wait'])

cluster_metrics['estimated_service_rate'] = (
    (cluster_metrics['avg_wait'] * cluster_metrics['arrival_rate'] + 
     np.sqrt((cluster_metrics['avg_wait'] * cluster_metrics['arrival_rate'])**2 + 
             4 * cluster_metrics['avg_wait'] * cluster_metrics['arrival_rate'])) 
    / (2 * cluster_metrics['avg_wait'])
)

cluster_metrics['traffic_intensity'] = cluster_metrics['arrival_rate'] / cluster_metrics['death_rate']

qos_results = []
for _, row in cluster_metrics.iterrows():
    cluster = row['cluster']
    arrival_rate = row['arrival_rate']
    estimated_service_rate = row['estimated_service_rate']
    
    qos_estimates = {}
    for x in thresholds:
        if estimated_service_rate > arrival_rate:
            qos = 1 - ((arrival_rate / estimated_service_rate) * np.exp(-(estimated_service_rate - arrival_rate) * x))
            qos_estimates[f"<= {x} mins"] = f"{qos:.1%}"
        else:
            qos_estimates[f"<= {x} mins"] = "N/A"
    
    result = {
        "Cluster": cluster,
        "Est Serv Rate": round(row['estimated_service_rate'], 3),
        "Est ρ": round(row['traffic_intensity'], 3),
        **qos_estimates
    }
    qos_results.append(result)

qos_df = pd.DataFrame(qos_results)
qos_df


/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/2043279047.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cluster_metrics = df.groupby('cluster').apply(
/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/2043279047.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cluster_metrics = df.groupby('cluster').apply(


,Cluster,Est Serv Rate,Est ρ,<= 5 mins,<= 10 mins,<= 15 mins,<= 20 mins,<= 25 mins,<= 30 mins
0,Weekday - 0:00 - 4:00,NaN,NaN,N/A,N/A,N/A,N/A,N/A,N/A
1,Weekday - 4:00 - 8:00,0.581,0.822,50.9%,70.7%,82.5%,89.6%,93.8%,96.3%
2,Weekday - 8:00 - 12:00,0.795,0.835,56.5%,77.4%,88.2%,93.9%,96.8%,98.3%
3,Weekday - 12:00 - 16:00,0.835,0.806,64.1%,84.0%,92.9%,96.8%,98.6%,99.4%
4,Weekday - 16:00 - 20:00,0.802,0.770,69.4%,87.8%,95.2%,98.1%,99.2%,99.7%
5,Weekday - 20:00 - 0:00,0.059,0.124,90.4%,92.6%,94.3%,95.6%,96.6%,97.4%
6,Weekend - 0:00 - 4:00,NaN,NaN,N/A,N/A,N/A,N/A,N/A,N/A
7,Weekend - 4:00 - 8:00,0.566,0.773,59.3%,78.6%,88.7%,94.1%,96.9%,98.4%
8,Weekend - 8:00 - 12:00,0.708,0.783,63.6%,83.1%,92.1%,96.3%,98.3%,99.2%
9,Weekend - 12:00 - 16:00,0.899,0.813,65.0%,84.9%,93.5%,97.2%,98.8%,99.5%


In [49]:
cluster_metrics

,cluster,death_rate,avg_wait,arrival_rate,estimated_service_rate,traffic_intensity
0,Weekday - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN
1,Weekday - 4:00 - 8:00,0.581154,7.969442,0.477957,0.581154,0.822427
2,Weekday - 8:00 - 12:00,0.794534,6.391859,0.663823,0.794534,0.835487
3,Weekday - 12:00 - 16:00,0.835245,4.975662,0.673247,0.835245,0.806047
4,Weekday - 16:00 - 20:00,0.802446,4.171182,0.617855,0.802446,0.769964
5,Weekday - 20:00 - 0:00,0.059098,2.401267,0.007344,0.059098,0.124274
6,Weekend - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN
7,Weekend - 4:00 - 8:00,0.565625,6.024855,0.437302,0.565625,0.773130
8,Weekend - 8:00 - 12:00,0.707576,5.109997,0.554278,0.707576,0.783349
9,Weekend - 12:00 - 16:00,0.899053,4.823921,0.730595,0.899053,0.812627


In [24]:
qos_df.to_csv("data-p3/mm1_qos_results.csv",index=False)